In [21]:
import pandas as pd
import numpy as np
import gc
import os
import warnings
warnings.filterwarnings("ignore")

import joblib


data_path = "../../3. 경서경욱_데이터 및 모델 세이브 파일/dataset/"
model_path = "../../3. 경서경욱_데이터 및 모델 세이브 파일/model/"

In [22]:
model = joblib.load(model_path+'svd.pkl')

In [23]:
data = pd.read_csv(data_path + "custom_dataset/collab_filtering_valset.csv", index_col = 0)
product = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_04_PD_CLAC.csv")
pro_list = list(product.pd_nm.unique())

# 장바구니 분석을 통한 상품추천
## - class 설명 -
#### def __init__()
    1) 초기값  
      (1) self.model : 저장된 surprise의 svd모델
      (2) self.items : custID|item|구매횟수 데이터프레임
      (3) self.pro_list : 모든 상품 리스트
      (4) self.custID : 고객식별번호
               
      (5) def get_unbought()
           -> 전체 상품 중 custID가 구매하지 않았던 상품리스트 return

    3) def recomm_item_by_surprise()
        -> make_basket()에서 생성된 장바구니(basket)을 통해 장바구니 분석 진행
        -> 지지도와 해당하는 장바구니를 return
    4) make_basket으로 장바구니 초깃값 생성 : self.basket
    5) make_support_score로 초깃값 생성 : self.support
 
#### def recomm_item_by_surprise()
    1) 초깃값에서 생성된 support_score를 바탕으로 입력된 product와 지지도가 형성된 상품 추천

In [51]:
class collab_filtering():
    def __init__(self, model, items, pro_list, custID):
        self.model = model
        self.items = items
        self.pro_list = pro_list
        self.custID = custID
    
        def get_unbought(self, items, pro_list, custID):
            # 특정 custId가 구매내역이 있는 상품
            bought_item = items[items['custID']== custID]['item'].tolist()

            # 특정 custId가 구매내역이 없 상품
            unbought_item = [item for item in pro_list if item not in bought_item]

            # 전체 상품 수, 구매내역 있는 상품 수, 구매내역 없는 상품 수
            total_product_cnt = len(pro_list)
            bought_cnt = len(bought_item)
            unbought_cnt = len(unbought_item)

            print(f"전체 상품 수: {total_product_cnt}, 구매 내역있는 상품 수: {bought_cnt}, 추천 대상 상품 수: {unbought_cnt}")
            print("\n")
            return unbought_item
        
        self.unbought_item = get_unbought(self, self.items, self.pro_list, self.custID)

    def recomm_item_by_surprise(self, top_n=10):

        # 아직 구매경험이 없은 상품에 대한 호감도 예측: prediction 객체 생성
        predictions = []    
        for item in self.unbought_item:
            predictions.append(self.model.predict(str(self.custID), str(item)))

        # 리스트 내의 prediction 객체의 est를 기준으로 내림차순 정렬
        def sortkey_est(pred):
            return pred.est

        predictions.sort(key=sortkey_est, reverse=True) # key에 리스트 내 객체의 정렬 기준을 입력

        # 상위 top_n개의 prediction 객체
        top_predictions = predictions[:top_n]

        # 상품, 예측 호감도 출력
        print(f"Top-{top_n} 추천 상품 리스트")

        for pred in top_predictions:
            item_nm = pred.iid
            item_ratings = pred.est
            print(f"{item_nm}: {item_ratings:.2f}")


In [52]:
if __name__ == "__main__":
    a = collab_filtering(model, data, pro_list, 'M905357061')
    a.recomm_item_by_surprise()

전체 상품 수: 1932, 구매 내역있는 상품 수: 90, 추천 대상 상품 수: 1842


Top-10 추천 상품 리스트
숙박: 0.12
반건오징어: 0.11
처방사료: 0.10
플라스틱서랍장: 0.10
기타광학기기: 0.09
여아코스튬: 0.09
브로치: 0.08
핸드폰충전: 0.08
신문/잡지: 0.08
유자/모과: 0.08
